# Scraping

We are using the info from this ![web site](https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoodpop.htm]) to complete our data.

In [1]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
from tabulate import tabulate

### To start, we save the URL of the website we will be scraping from

In [2]:
url_population = "https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoodpop.htm"

In [3]:
response = requests.get(url_population)
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
population = soup('table', {"class": 'light_table right'})

### Now, we look for the tags inside, checking the first element

In [5]:
data = population[0].find_all('tr')

In [6]:
data[0]

<tr>
<th id="b" rowspan="1">Borough</th>
<th id="c" rowspan="1">region</th>
<th id="m">Males</th>
<th id="f">Females</th>
<th id="p">Total Population</th>
</tr>

In [7]:
data[0].find_all('th')

[<th id="b" rowspan="1">Borough</th>,
 <th id="c" rowspan="1">region</th>,
 <th id="m">Males</th>,
 <th id="f">Females</th>,
 <th id="p">Total Population</th>]

### We look for the data inside our dataframe 

In [8]:
population2 = []
for row in data:
    population2.append([cell.text for cell in row.find_all()])

In [9]:
population_final = pd.DataFrame(population2)
population_final.columns = population_final.iloc[0]
population_final = population_final.iloc[1:]

In [10]:
population_final [:5]

,Borough,region,Males,Females,Total Population
1,Bronx,"Riverdale, Fieldston & Kingsbridge","51,598","61,481","113,079"
2,,"Wakefield, Williamsbridge & Woodlawn","65,216","78,387","143,604"
3,,"Co-op City, Pelham Bay & Schuylerville","55,037","65,204","120,241"
4,,"Pelham Parkway, Morris Park & Laconia","60,974","67,982","128,956"
5,,"Belmont, Crotona Park East & East Tremont","77,119","89,293","166,411"


A table with empty cells is obtained.

### We work to include in the empty cells the values we are looking for.

#### Firstly, we replace empty cells by "nan"

In [11]:
population_final = population_final.replace(r'^\s*$', np.nan, regex=True)

In [12]:
population_final[:12]

,Borough,region,Males,Females,Total Population
1,Bronx,"Riverdale, Fieldston & Kingsbridge","51,598","61,481","113,079"
2,NaN,"Wakefield, Williamsbridge & Woodlawn","65,216","78,387","143,604"
3,NaN,"Co-op City, Pelham Bay & Schuylerville","55,037","65,204","120,241"
4,NaN,"Pelham Parkway, Morris Park & Laconia","60,974","67,982","128,956"
5,NaN,"Belmont, Crotona Park East & East Tremont","77,119","89,293","166,411"
6,NaN,"Bedford Park, Fordham North & Norwood","63,169","68,921","132,090"
7,NaN,"Morris Heights, Fordham South & Mount Hope","65,682","72,967","138,648"
8,NaN,"Concourse, Highbridge & Mount Eden","68,152","75,978","144,129"
9,NaN,"Castle Hill, Clason Point & Parkchester","88,007","100,193","188,201"
10,NaN,"Hunts Point, Longwood & Melrose","81,666","79,759","161,425"


#### In second place, we define a function to substitute the name of some values for the correct ones.

In [13]:
def neighbourhood (Borough):
    if Borough == "Kings (Brooklyn)":
        return "Brooklyn" 
    elif Borough == "New York (Manhattan)":
        return "Manhattan" 
    elif Borough == "Bronx":
        return "Bronx"
    elif Borough == "Queens":
        return "Queens" 
    elif Borough == "Richmond (Staten Island)":
        return "Staten Island"

In [14]:
population_final ["Borough"] = population_final["Borough"].apply(neighbourhood)

#### Finaly, we define new dataframes for each value we want to include in the table.

In [15]:
bronx = population_final.loc[0:10].replace(np.nan,"Bronx")

In [16]:
brooklyn = population_final.loc[11:29].replace(np.nan,"Brooklyn")

In [17]:
manhattan = population_final.loc[30:39].replace(np.nan,"Manhattan")

In [18]:
queens = population_final.loc[40:53].replace(np.nan,"Queens")

In [19]:
staten_island = population_final.loc[54:56].replace(np.nan,"Staten Island")

In [20]:
NYC_population = pd.concat([bronx, brooklyn, manhattan, queens, staten_island])

### Once our dataset is clean, we reset the index

In [21]:
NYC_population.head()

,Borough,region,Males,Females,Total Population
1,Bronx,"Riverdale, Fieldston & Kingsbridge","51,598","61,481","113,079"
2,Bronx,"Wakefield, Williamsbridge & Woodlawn","65,216","78,387","143,604"
3,Bronx,"Co-op City, Pelham Bay & Schuylerville","55,037","65,204","120,241"
4,Bronx,"Pelham Parkway, Morris Park & Laconia","60,974","67,982","128,956"
5,Bronx,"Belmont, Crotona Park East & East Tremont","77,119","89,293","166,411"


In [22]:
NYC_population.shape

(55, 5)

### Finally, we export our cleaned data to used in other files.

In [ ]:
NYC_population.to_csv("./output/NYC_population.csv", index = False)